# BIG DATA ANALYSIS: Pandas BuiltIn Plot적용

### 탐색적 데이터 분석을 이용해 각각의 Feature들의 중요성 파악 및 새로운 Feature 생성
---

In [ ]:
import sys
!{sys.executable} -m pip install pandas matplotlib

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

## 1. 데이터 확보 및 결측값 처리

In [ ]:
df = pd.read_csv("data/train.csv")
df.info()


In [ ]:
#한글 폰트를 사용할 시! 외우지 마시고 그때 그때 복사해서 사용!
import platform
from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus'] = False
if platform.system() == 'Darwin':
    rc('font', family = 'AppleGothic')
elif platform.system() == 'Windows':
    path ='C:/Windows/Fonts/malgun.ttf'
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print("Can't find your system.")



## 1. 간단한 Box plot으로 Numeric 속성들에 대한 분포 확인

- count: 객체 수
- mean: 평균
- std: 표준편차
- min: 최소값 (이상치-outlier 포함)
- 25%: 전체 데이터를 순서대로 정렬 했을시, 25%지점 (1/4)에 해당하는 값
- 50%: 전체 데이터를 순서대로 정렬 했을시, 50%지점 (1/2)에 해당하는 값 (중위값)
- 70%: 전체 데이터를 순서대로 정렬 했을시, 70%지점 (3/4)에 해당하는 값
- max: 최대값 (이상치-outlier 포함)
<img src="data/boxplot.png"/>

In [ ]:
df.plot(kind='box')

### 1-1. 나이 속성에 대한 정보 확인하기

In [ ]:
df['나이'].describe()

In [ ]:

#pandas내에 정의된 plot 함수 활용
df['나이'].plot(kind='box')

### 1-2. 요금 속성에 대한 정보 확인하기

In [ ]:
df['요금'].plot(kind='box')

In [ ]:
df["형제자매"].plot(kind='box')

In [ ]:
df["부모"].plot(kind='box')

In [ ]:
#사실 범주형 데이터로 보는게 맞음!
df['부모'].value_counts()

## 2. Pandas Group by를 이용하여, 집단에 대한 통계량확인

### 2-1. 1-depth 분석

In [ ]:
# 성별 따른 전체 데이터 분류
print(df.groupby('성별')['성별'].count())
df.groupby('성별')['성별'].count().plot(kind='bar')

In [ ]:
# 생존 여부에 따른 전체 데이터 분류
print(df.groupby('생존')['생존'].count())
df.groupby('생존')['생존'].count().plot(kind='bar')

In [ ]:
# 객실등급 여부에 따른 전체 데이터 분류
print(df.groupby('객실등급')['객실등급'].count())
df.groupby('객실등급')['객실등급'].count().plot(kind='bar')

### 2-2. 2-depth 분석

In [ ]:
# 객실등급-성별에 따른 분류
print(df.groupby(['객실등급','성별'])['성별'].count())
print(df.groupby(['객실등급','성별'])['성별'].count().unstack())

df.groupby(['객실등급','성별'])['성별'].count().unstack().plot(kind='bar')
# df.groupby(['생존','성별'])['성별'].count().unstack().plot(kind='bar',stacked=True)

In [ ]:
# 좀 더 깊게 들어가서, 남성, 여성, 어린이 항목으로 나눠 볼 수 있지 않을까?

In [ ]:
def titanic_children(passenger):
    age , sex = passenger
    if age <16:
        return 'child'
    elif age>40:
        return 'senior'
    else:
        return sex

df['사람'] = df[['나이','성별']].apply(titanic_children,axis=1)
df.head()

In [ ]:
# 생존-성별에 따른 분류
print(df.groupby(['객실등급','사람'])['사람'].count())
print(df.groupby(['객실등급','사람'])['사람'].count().unstack())

df.groupby(['객실등급','사람'])['사람'].count().unstack().plot(kind='bar')
# df.groupby(['생존','성별'])['성별'].count().unstack().plot(kind='bar',stacked=True)

### 2-3. EDA를 통한 결측값 처리

나이 속성의 결측 값을 전체 나이의 평균 또는 남녀 나이의 평균으로 했는데, 조금 더 구체화 할 수 있지 않을까?

In [ ]:
df.groupby(['객실등급'])['나이'].plot(kind='hist',bins=10,alpha=0.3,figsize=(20,10))
plt.legend()

## 3. 생존과 연관이 있는 속성 파악하기!

### 3-1. 나이와 생존이 연관이 있을까?

In [ ]:
print(df.groupby(['생존'])['나이'].mean())
df.groupby(['생존'])['나이'].mean().plot(kind='bar')

In [ ]:
df.groupby(['생존','사람'])['사람'].count().unstack().plot(kind='bar')

### 3-2. 요금과 생존이 연관이 있을까?

In [ ]:

print(df.groupby(['생존'])['요금'].mean())
df.groupby(['생존'])['요금'].mean().plot(kind='bar')


### 3-3. 성별과 생존이 연관이 있을까?

In [ ]:
# 생존-성별에 따른 분류
print(df.groupby(['생존','성별'])['성별'].count())
print(df.groupby(['생존','성별'])['성별'].count().unstack())

df.groupby(['생존','성별'])['성별'].count().unstack().plot(kind='bar')
# df.groupby(['생존','성별'])['성별'].count().unstack().plot(kind='bar',stacked=True)

### 3-4. 객실 등급과 생존이 연관이 있을까?

In [ ]:

print(df.groupby(['생존','객실등급'])['객실등급'].count().unstack())

df.groupby(['생존','객실등급'])['객실등급'].count().unstack().plot(kind='bar')
# df.groupby(['생존','성별'])['성별'].count().unstack().plot(kind='bar',stacked=True)

### 3-5. 승선항과 생존이 연관이 있을까?

In [ ]:

print(df.groupby(['생존','승선항'])['승선항'].count().unstack())

df.groupby(['생존','승선항'])['승선항'].count().unstack().plot(kind='bar')
# df.groupby(['생존','성별'])['성별'].count().unstack().plot(kind='bar',stacked=True)

In [ ]:
#그냥 S항에서 승선한 사람이 많을 뿐..
df.groupby('승선항')['승선항'].count().plot(kind='bar')